In [1]:
%matplotlib inline
import os  
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import statsmodels.api as sm  
# import seaborn as sns 
# sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

In [15]:
def get_closest_training(capacity ,dest_city, time, lh_ind, df):
    """ 
     This finction takes in the prediction DF and filtered training DF
    Then applies further filters to get an average for the arrival times.
    1. All Past Dates
    2. Same City Capacity
    3. Same City Capacity & Day of week
    4. Same City Capacity & Day of week & time
    Then checks the length od the 4 data frames to find the smallest to use
    """
    
    # Subset 1
    subset1 = df.loc[(df["cat_s_plane_capacity"] == capacity)]
    # Subset 2
    subset2 = subset1.loc[(subset1["cat_longhaul_ind"] == lh_ind)]
    # Subset 3
    subset3 = subset2.loc[(subset2["cat_i_city"] == dest_city)]
    
    # Subset 5
    subset4 = subset3.loc[(subset3["cat_sdt_hour"] == time)]
    
    if (len(subset4.index) > 0):
        return subset4
    elif (len(subset3.index) > 0):
        return subset3
    elif (len(subset2.index) > 0):
        return subset2
    elif (len(subset1.index) > 0):
        return subset1
    else:
        return df
    

In [3]:
def populate_means(df):
    df_new = df[['num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt',
       'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt',
       'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt',
       'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',
       'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt',
       'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',
       'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt',
       'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt',
       'num_pax_240plus_mins_before_sdt']]

    df_mean = pd.DataFrame(df_new.mean(axis= 0).astype(int))
    df_t = df_mean.transpose()
    return df_t




In [4]:
########################################################
# 2. Define variables which we will use later in the script
########################################################
#s_roor_folder = "C:\Users\piush\Desktop\Dataset\data\"
#s_train_file = 'train.csv' 
l_parse_date_cols = ['dt_prediction_date', 'dt_target_date', 'dt_flight_date']
l_target_cols = ['num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt', 'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt', 'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt', 'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',  'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt', 'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',  'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt', 'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt', 'num_pax_240plus_mins_before_sdt']

In [5]:
# Set some parameters to get good visuals - style to ggplot and size to 15,10
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)

In [6]:
df = pd.read_csv((r"C:\Users\piush\Desktop\Dataset\data_20161116\FullSet.csv", parse_dates = l_parse_date_cols)
df = df.set_index('id')

In [8]:
print ("Shape of training set:", df.shape, "\n")
print ("Column Headers:", list(df.columns.values), "\n")
print (df.dtypes)

Shape of training set: (136194, 54) 

Column Headers: ['dt_prediction_date', 'dt_target_date', 'Source', 's_model_type', 'cat_case_type', 'cat_i_flightno', 'dt_flight_date', 'num_flight_year', 'num_flight_month', 'num_flight_weekofyear', 'num_flight_dayofweek', 'cat_sdt_hour', 'cat_i_airport', 'cat_i_city', 'cat_destination_group_id', 'cat_longhaul_ind', 'num_airport_other_ops', 'num_airport_other_flights', 'num_city_other_ops', 'num_city_other_flights', 'cat_s_plane_capacity', 'cat_flight_class_type_id', 'ord_leisure', 'ord_irish_residents', 'ord_trip_duration', 'ord_female', 'ord_party_size', 'ord_bag_checkin', 'ord_arrive_by_car', 'num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt', 'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt', 'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt', 'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt', 'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt', 'num_pax

In [7]:

df['num_pax_000_014_mins_before_sdt'].fillna(0, inplace=True)   
df['num_pax_015_029_mins_before_sdt'].fillna(0, inplace=True)           
df['num_pax_030_044_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_045_059_mins_before_sdt'].fillna(0, inplace=True)         
df['num_pax_060_074_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_075_089_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_090_104_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_105_119_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_120_134_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_135_149_mins_before_sdt'].fillna(0, inplace=True)          
df['num_pax_150_164_mins_before_sdt'].fillna(0, inplace=True)           
df['num_pax_165_179_mins_before_sdt'].fillna(0, inplace=True)         
df['num_pax_180_194_mins_before_sdt'].fillna(0, inplace=True)         
df['num_pax_195_209_mins_before_sdt'].fillna(0, inplace=True)         
df['num_pax_210_224_mins_before_sdt'].fillna(0, inplace=True)           
df['num_pax_225_239_mins_before_sdt'].fillna(0, inplace=True)           
df['num_pax_240plus_mins_before_sdt'].fillna(0, inplace=True) 
    
df.head(10)

,dt_prediction_date,dt_target_date,Source,s_model_type,cat_case_type,cat_i_flightno,dt_flight_date,num_flight_year,num_flight_month,num_flight_weekofyear,...,num_pax_225_239_mins_before_sdt,num_pax_240plus_mins_before_sdt,TotalOnFlight,National Holiday,Not National Holiday,Rugby - Home,Rugby Away,PrimaryShcoolsOff,SecondarySchoolsOff,HolidaySeason
id,,,,,,,,,,,,,,,,,,,,,
10099,2015-07-17,2015-07-18,TrainData,01_day,Expl,744,2015-07-15,2015,7,29,...,0.0,0.0,168,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,TrainData,01_day,Expl,744,2015-07-16,2015,7,29,...,0.0,0.0,154,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,TrainData,01_day,Expl,744,2015-07-17,2015,7,29,...,0.0,1.0,151,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,TrainData,01_day,Target,744,2015-07-18,2015,7,29,...,2.0,3.0,177,0,0,0,0,1,1,0
101,2013-03-14,2014-03-14,TrainData,52_weeks,Target,10407,2014-03-14,2014,3,11,...,0.0,0.0,211,0,0,0,0,0,0,0
1010,2013-12-30,2014-07-01,TrainData,26_weeks,Target,8212,2014-07-01,2014,7,27,...,0.0,0.0,7,0,0,0,0,1,1,0
10100,2015-07-17,2015-07-18,TrainData,01_day,Expl,1117,2013-07-20,2013,7,29,...,0.0,0.0,160,0,0,0,0,1,1,0
10100,2015-07-17,2015-07-18,TrainData,01_day,Expl,1117,2014-01-18,2014,1,3,...,0.0,0.0,145,0,0,0,0,0,0,0
10100,2015-07-17,2015-07-18,TrainData,01_day,Expl,1117,2014-07-19,2014,7,29,...,0.0,0.0,170,0,0,0,0,1,1,0


In [46]:
# Have a look at the number of prediction dates 

df["dt_prediction_date"],df["cat_case_type"],df["Source"]

key_cols = ['dt_prediction_date','cat_case_type','Source']

df.groupby(key_cols).size()

dt_prediction_date  cat_case_type  Source   
2013-01-01          Target         TrainData      1
2013-01-02          Expl           TrainData      6
                    Target         TrainData      1
2013-01-04          Expl           TrainData     11
                    Target         TrainData      4
2013-01-05          Target         TrainData      1
2013-01-07          Expl           TrainData     12
                    Target         TrainData      3
2013-01-09          Expl           TrainData      7
                    Target         TrainData      1
2013-01-10          Expl           TrainData     26
                    Target         TrainData      3
2013-01-11          Expl           TrainData      8
                    Target         TrainData      3
2013-01-12          Target         TrainData      1
2013-01-13          Expl           TrainData     13
                    Target         TrainData      1
2013-01-15          Target         TrainData      1
2013-01-16         

In [8]:
df['s_model_type'] = df['s_model_type'].str.strip('_weeks').convert_objects(convert_numeric=True)

In [9]:
Source_g = {"TrainData" : 1, "TestData" : 2}
df["Source"] = df["Source"].apply(lambda s: Source_g.get(s))

In [10]:
cat_case_type_g = {"Target" : 1, "Expl" : 0}
df["cat_case_type"] = df["cat_case_type"].apply(lambda s: cat_case_type_g.get(s))

In [11]:
'051 - 100' '151 - 200' '101 - 150' '201 - 300' '300+' '001 - 050'
cat_s_plane_capacity_g = {"001 - 050" : 1, "051 - 100" : 2,"101 - 150" : 3, "151 - 200" : 4,"201 - 300" : 5, "300+" : 6}
df["cat_s_plane_capacity"] = df["cat_s_plane_capacity"].apply(lambda s: cat_s_plane_capacity_g.get(s))

In [12]:
df['s_model_type'] = df.loc[:, "s_model_type"].fillna(0.142)

In [19]:
df.head(5)

,dt_prediction_date,dt_target_date,Source,s_model_type,cat_case_type,cat_i_flightno,dt_flight_date,num_flight_year,num_flight_month,num_flight_weekofyear,...,num_pax_225_239_mins_before_sdt,num_pax_240plus_mins_before_sdt,TotalOnFlight,National Holiday,Not National Holiday,Rugby - Home,Rugby Away,PrimaryShcoolsOff,SecondarySchoolsOff,HolidaySeason
id,,,,,,,,,,,,,,,,,,,,,
10099,2015-07-17,2015-07-18,1,0.142,0,744,2015-07-15,2015,7,29,...,0.0,0.0,168,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,1,0.142,0,744,2015-07-16,2015,7,29,...,0.0,0.0,154,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,1,0.142,0,744,2015-07-17,2015,7,29,...,0.0,1.0,151,0,0,0,0,1,1,0
10099,2015-07-17,2015-07-18,1,0.142,1,744,2015-07-18,2015,7,29,...,2.0,3.0,177,0,0,0,0,1,1,0
101,2013-03-14,2014-03-14,1,52.000,1,10407,2014-03-14,2014,3,11,...,0.0,0.0,211,0,0,0,0,0,0,0


In [37]:

# Slit the data based on the cat case type 
# targetDF = df.loc[(df["cat_case_type"] == 1) & df['dt_prediction_date'] == '2016-05-26']
targetDF = df.loc[(df["TotalOnFlight"] == 0) & (df["dt_prediction_date"] == '2016-05-26')]
trainingDF = df.loc[(df["TotalOnFlight"] != 0)]
# Create data frame to hold the average predictions
avg_pred = targetDF

# DefineKeyColumns

# For each row in the targetDF (ie what we are trying to predict)
#  1. Get the historical Flight data
for index, row in avg_pred.iterrows():
# Filter the taining data based on A past fligts or B past predictions
#  Historical flights:
    mask = (trainingDF['dt_flight_date'] <= row['dt_prediction_date'].date())
#  Historical predictions... 
#     mask = (trainingDF['dt_prediction_date'] <= row['dt_prediction_date'].date())

# df_date is filtered to the knowen dates: 
    df_date = trainingDF.loc[mask]
    
# Call function to calculate the average ( modle 1 )
    df_best_match = get_closest_training(row['cat_s_plane_capacity'], row['cat_i_city']
                                        ,row['cat_sdt_hour'], row['cat_longhaul_ind'] ,df_date)
# Pass the df_date and keycol from row['dt_prediction_date']    
    Best_Fit = populate_means(df_best_match).astype('int')
    
    avg_pred.loc[index, "num_pax_000_014_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_000_014_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_015_029_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_015_029_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_030_044_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_030_044_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_045_059_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_045_059_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_060_074_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_060_074_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_075_089_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_075_089_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_090_104_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_090_104_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_105_119_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_105_119_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_120_134_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_120_134_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_135_149_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_135_149_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_150_164_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_150_164_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_165_179_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_165_179_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_180_194_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_180_194_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_195_209_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_195_209_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_210_224_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_210_224_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_225_239_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_225_239_mins_before_sdt"]
    avg_pred.loc[index, "num_pax_240plus_mins_before_sdt"] = Best_Fit.iloc[0]["num_pax_240plus_mins_before_sdt"]


In [39]:
avg_pred

,dt_prediction_date,dt_target_date,Source,s_model_type,cat_case_type,cat_i_flightno,dt_flight_date,num_flight_year,num_flight_month,num_flight_weekofyear,...,num_pax_225_239_mins_before_sdt,num_pax_240plus_mins_before_sdt,TotalOnFlight,National Holiday,Not National Holiday,Rugby - Home,Rugby Away,PrimaryShcoolsOff,SecondarySchoolsOff,HolidaySeason
id,,,,,,,,,,,,,,,,,,,,,
14906,2016-05-26,2016-05-27,2,0.142,1,700,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14907,2016-05-26,2016-05-27,2,0.142,1,140,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14908,2016-05-26,2016-05-27,2,0.142,1,335,2016-05-27,2016,5,21,...,0.0,1.0,0,0,0,0,0,0,0,0
14909,2016-05-26,2016-05-27,2,0.142,1,239,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14910,2016-05-26,2016-05-27,2,0.142,1,1233,2016-05-27,2016,5,21,...,0.0,2.0,0,0,0,0,0,0,0,0
14911,2016-05-26,2016-05-27,2,0.142,1,1313,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14912,2016-05-26,2016-05-27,2,0.142,1,696,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14913,2016-05-26,2016-05-27,2,0.142,1,1082,2016-05-27,2016,5,21,...,0.0,0.0,0,0,0,0,0,0,0,0
14914,2016-05-26,2016-05-27,2,0.142,1,864,2016-05-27,2016,5,21,...,0.0,4.0,0,0,0,0,0,0,0,0


In [24]:
Best_Fit.columns

Index(['num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt',
       'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt',
       'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt',
       'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',
       'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt',
       'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',
       'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt',
       'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt',
       'num_pax_240plus_mins_before_sdt'],
      dtype='object')

In [20]:
avg_pred.to_csv("A.csv", index = False)

In [82]:
sample = avg_pred.loc[(avg_pred["dt_prediction_date"] == '2016-05-26') ]
sample[['num_pax_015_029_mins_before_sdt', 'num_pax_030_044_mins_before_sdt',
       'num_pax_045_059_mins_before_sdt', 'num_pax_060_074_mins_before_sdt',
       'num_pax_075_089_mins_before_sdt', 'num_pax_090_104_mins_before_sdt',
       'num_pax_105_119_mins_before_sdt', 'num_pax_120_134_mins_before_sdt',
       'num_pax_135_149_mins_before_sdt', 'num_pax_150_164_mins_before_sdt',
       'num_pax_165_179_mins_before_sdt', 'num_pax_180_194_mins_before_sdt',
       'num_pax_195_209_mins_before_sdt', 'num_pax_210_224_mins_before_sdt',
       'num_pax_225_239_mins_before_sdt', 'num_pax_240plus_mins_before_sdt']]

,num_pax_015_029_mins_before_sdt,num_pax_030_044_mins_before_sdt,num_pax_045_059_mins_before_sdt,num_pax_060_074_mins_before_sdt,num_pax_075_089_mins_before_sdt,num_pax_090_104_mins_before_sdt,num_pax_105_119_mins_before_sdt,num_pax_120_134_mins_before_sdt,num_pax_135_149_mins_before_sdt,num_pax_150_164_mins_before_sdt,num_pax_165_179_mins_before_sdt,num_pax_180_194_mins_before_sdt,num_pax_195_209_mins_before_sdt,num_pax_210_224_mins_before_sdt,num_pax_225_239_mins_before_sdt,num_pax_240plus_mins_before_sdt
id,,,,,,,,,,,,,,,,
14906,0.0,1.0,5.0,15.0,30.0,24.0,19.0,14.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
14907,0.0,1.0,1.0,10.0,7.0,8.0,8.0,2.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0
14908,0.0,1.0,2.0,9.0,10.0,10.0,9.0,5.0,2.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0
14909,0.0,0.0,2.0,7.0,16.0,4.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14910,0.0,1.0,4.0,4.0,9.0,10.0,10.0,5.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0
14911,0.0,1.0,8.0,17.0,33.0,29.0,22.0,21.0,20.0,9.0,3.0,1.0,0.0,0.0,0.0,0.0
14912,0.0,0.0,4.0,12.0,37.0,30.0,36.0,23.0,12.0,6.0,7.0,4.0,0.0,0.0,0.0,0.0
14913,0.0,2.0,9.0,18.0,23.0,25.0,21.0,16.0,10.0,6.0,4.0,2.0,1.0,0.0,0.0,0.0
14914,0.0,0.0,10.0,14.0,17.0,19.0,12.0,13.0,3.0,8.0,1.0,1.0,4.0,0.0,0.0,7.0


In [30]:
targetDF.head(5)

,dt_prediction_date,dt_target_date,Source,s_model_type,cat_case_type,cat_i_flightno,dt_flight_date,num_flight_year,num_flight_month,num_flight_weekofyear,...,num_pax_225_239_mins_before_sdt,num_pax_240plus_mins_before_sdt,TotalOnFlight,National Holiday,Not National Holiday,Rugby - Home,Rugby Away,PrimaryShcoolsOff,SecondarySchoolsOff,HolidaySeason
id,,,,,,,,,,,,,,,,,,,,,
10099,2015-07-17,2015-07-18,1,0.142,1,744,2015-07-18,2015,7,29,...,2.0,3.0,177,0,0,0,0,1,1,0
101,2013-03-14,2014-03-14,1,52.000,1,10407,2014-03-14,2014,3,11,...,0.0,0.0,211,0,0,0,0,0,0,0
1010,2013-12-30,2014-07-01,1,26.000,1,8212,2014-07-01,2014,7,27,...,0.0,0.0,7,0,0,0,0,1,1,0
10100,2015-07-17,2015-07-18,1,0.142,1,1117,2015-07-18,2015,7,29,...,0.0,0.0,176,0,0,0,0,1,1,0
10101,2015-07-17,2015-07-18,1,0.142,1,604,2015-07-18,2015,7,29,...,2.0,1.0,170,0,0,0,0,1,1,0


In [ ]:
df1 = pd.DataFrame(columns = 'num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt',
       'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt',
       'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt',
       'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',
       'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt',
       'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',
       'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt',
       'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt',
       'num_pax_240plus_mins_before_sdt')
df1.loc[df_t.index[0]] = df_t.iloc[0]

In [ ]:
# Now that we have relevent Flights 
# Look for similar flights

key_cols = ['cat_i_flightno','num_flight_year','num_flight_month', 'num_flight_weekofyear',
            'num_flight_dayofweek','cat_sdt_hour','cat_longhaul_ind','cat_s_plane_capacity',
            'ord_bag_checkin', 'ord_leisure', 'cat_i_city','cat_destination_group_id']

cat_longhaul_ind
cat_i_city

len()